In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
import numpy as np
import keras
# Generate  data
x_train = np.random.random((100, 1))
y_train = np.random.random((100, 1))

x_test = np.random.random((20, 1))
y_test = np.random.random((20, 1))

model = Sequential()
model.add(Embedding(1000, output_dim=256))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=16)


Epoch 1/10
100/100 [==============================] - 1s - loss: 0.6933 - acc: 0.0000e+00     
Epoch 2/10
100/100 [==============================] - 0s - loss: 0.6914 - acc: 0.0000e+00     
Epoch 3/10
100/100 [==============================] - 0s - loss: 0.6935 - acc: 0.0000e+00     
Epoch 4/10
100/100 [==============================] - 0s - loss: 0.6915 - acc: 0.0000e+00     
Epoch 5/10
100/100 [==============================] - 0s - loss: 0.6906 - acc: 0.0000e+00     
Epoch 6/10
100/100 [==============================] - 0s - loss: 0.6912 - acc: 0.0000e+00     
Epoch 7/10
100/100 [==============================] - 0s - loss: 0.6913 - acc: 0.0000e+00     
Epoch 8/10
100/100 [==============================] - 0s - loss: 0.6920 - acc: 0.0000e+00     
Epoch 9/10
100/100 [==============================] - 0s - loss: 0.6911 - acc: 0.0000e+00     
Epoch 10/10
16/20 [=======================>......] - ETA: 0s

In [17]:
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf
from keras.layers import AvgPool1D

class MyMeanPool(Layer):
    def __init__(self, axis, **kwargs):
        self.supports_masking = True
        self.axis = axis
        super(MyMeanPool, self).__init__(**kwargs)

    def compute_mask(self, input, input_mask=None):
        # need not to pass the mask to next layers
        return None

    def call(self, x, mask=None):
        if mask is not None:
            mask = K.repeat(mask, x.shape[-1])
            mask = tf.transpose(mask, [0,2,1])
            mask = K.cast(mask, K.floatx())
            x = x * mask
            return K.sum(x, axis=self.axis) / K.sum(mask, axis=self.axis)
        else:
            return K.mean(x, axis=self.axis)

    def compute_output_shape(self, input_shape):
        output_shape = []
        for i in range(len(input_shape)):
            if i!=self.axis:
                output_shape.append(input_shape[i])
        return tuple(output_shape)

ID_seq0 = [1, 0, 0, 0]
ID_seq1 = [1, 1, 0, 0]
ID_seq2 = [1, 1, 1, 0]
ID_seq3 = [1, 1, 1, 1]
data = np.array([ID_seq0, ID_seq1, ID_seq2, ID_seq3])

model = Sequential()
model.add(Embedding(5, 3, mask_zero=True))
model.add(MyMeanPool(1))

result = model.predict(data[3])
print(result)
print (model.summary())


[[ 0.03350211 -0.04146602 -0.00207807]
 [ 0.03350211 -0.04146602 -0.00207807]
 [ 0.03350211 -0.04146602 -0.00207807]
 [ 0.03350211 -0.04146602 -0.00207807]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, None, 3)           15        
_________________________________________________________________
my_mean_pool_8 (MyMeanPool)  (None, 3)                 0         
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________
None
